# Video Game Recomendation System

## Business understanding

## Data understanding

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Dataset from kaggle on [here](https://www.kaggle.com/datasets/cynthiarempel/amazon-us-customer-reviews-dataset?select=amazon_reviews_us_Digital_Video_Games_v1_00.tsv)

In [10]:
df_raw = pd.read_csv('./data/Digital_Video_Games.tsv', sep='\t', header=0, error_bad_lines=False)

### Columns
- **marketplace**: 2 letter country code of the marketplace where the review was written.
- **customer_id**: Random identifier that can be used to aggregate reviews written by a single author.
- **review_id**: The unique ID of the review.
- **productid**: The unique Product ID the review pertains to. In the multilingual dataset the reviews for the same product in different countries can be grouped by the same productid.
- **product_parent**: Random identifier that can be used to aggregate reviews for the same product.
- **product_title**: Title of the product.
- **product_category**: Broad product category that can be used to group reviews (also used to group the dataset into coherent parts).
- **star_rating**: The 1-5 star rating of the review.
- **helpful_votes**: Number of helpful votes.
- **total_votes**: Number of total votes the review received.
- **vine**: Review was written as part of the Vine program.
- **verified_purchase**: The review is on a verified purchase.
- **review_headline**: The title of the review.
- **review_body**: The review text.
- **review_date**: The date the review was written.

In [11]:
df_raw.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,Digital_Video_Games,2,2,3,N,N,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31
1,US,133437,R1WFOQ3N9BO65I,B00F4CEHNK,341969535,Xbox Live Gift Card,Digital_Video_Games,5,0,0,N,Y,Five Stars,Awesome,2015-08-31
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,Digital_Video_Games,5,0,0,N,Y,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31
3,US,113118,R3R14UATT3OUFU,B004RMK5QG,395682204,Playstation Plus Subscription,Digital_Video_Games,5,0,0,N,Y,Five Stars,Perfect,2015-08-31
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,Digital_Video_Games,5,0,0,N,Y,Five Stars,Awesome!,2015-08-31


In [12]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144724 entries, 0 to 144723
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        144724 non-null  object
 1   customer_id        144724 non-null  int64 
 2   review_id          144724 non-null  object
 3   product_id         144724 non-null  object
 4   product_parent     144724 non-null  int64 
 5   product_title      144724 non-null  object
 6   product_category   144724 non-null  object
 7   star_rating        144724 non-null  int64 
 8   helpful_votes      144724 non-null  int64 
 9   total_votes        144724 non-null  int64 
 10  vine               144724 non-null  object
 11  verified_purchase  144724 non-null  object
 12  review_headline    144722 non-null  object
 13  review_body        144722 non-null  object
 14  review_date        144721 non-null  object
dtypes: int64(5), object(10)
memory usage: 16.6+ MB


Some null values in the last 3 columns.

In [13]:
df_raw['marketplace'].unique()

array(['US'], dtype=object)

All reviews in dataset are from US customers

In [14]:
df_raw['customer_id'].nunique()

112891

112891 different customers 

In [15]:
df_raw['review_id'].nunique()

144724

Seems like there are no duplicate reviews in the set

In [16]:
df_raw['product_id'].nunique()

7939

7939 different products, but not all video games, found "Playstation Plus Subscription" in row 3, so all digitally sold products related video games, like subscriptions, seem to be included in the data set, will further look for other kind of product other than games and drop those.

In [17]:
df_raw['product_parent'].nunique()

7755

- **product_parent**: Random identifier that can be used to aggregate reviews for the same product.

seems like just another id but we have about 200 less data points in this column than product id

In [18]:
df_raw['product_title'].nunique()

6946

if i have 6946 different titles, i would expect the same amount of product_id, wonder if by droping everything that is not a game, like substriptions, would fix this discrepancy.

In [19]:
df_raw.iloc[3]['product_title']

'Playstation Plus Subscription'

It doesnt specify if the subscription is for 1, 3 or 12 months

<br>

Below i used key words to look for any products that is not a game
e.g
- Subscription
- Month
- Membership
- Card
- Network



In [23]:
Game_subs = df_raw[df_raw['product_title'].str.contains('Subscription')]
Game_subs['product_title'].unique()

array(['Playstation Plus Subscription', 'Xbox Live Subscription',
       '1 Month Subscription: EVE Online [Instant Access]',
       'PlayStation Now Subscription Twister Parent',
       'Webkinz 12 Month Deluxe Subscription [Online Game Code]'],
      dtype=object)

In [24]:
Sub_months = df_raw[df_raw['product_title'].str.contains('Month')]
Sub_months['product_title'].unique()

array(['1 Month Subscription: EVE Online [Instant Access]',
       'Xbox Live 24 Month Gold Membership - Xbox 360 Digital Code',
       'Disney Club Penguin 12 Month Membership Code with Free Amazon Exclusive Bonus Item',
       'Xbox Live 3-Month Gold + $100 Xbox Gift Card - Xbox 360 Digital Code',
       'Xbox Live 3-Month Gold + $5 Xbox Gift Card - Xbox 360 Digital Code',
       'Webkinz 12 Month Deluxe Subscription [Online Game Code]',
       'Xbox 12 Month Gold for Black Ops II - Xbox 360 Digital Code',
       '3-Month PS Plus + $10 PS Gift Card - PS3 / PS4 [Digital Code]',
       '3-Month PS Plus + $20 PS Gift Card - PS3 / PS4 [Digital Code]',
       'Xbox LIVE 12 Month Gold for Call of Duty: Ghost - Xbox 360 Digital Code',
       'Curse Premium - 1 Month [Download]',
       'GameBattles Premium Access - 3 Months [Online Game Code]',
       '3-Month PS Plus + $50 PS Gift Card - PS3 / PS4 [Digital Code]',
       'Xbox Live 12-Month Gold + $25 Xbox Gift Card - Xbox 360 Digital Code

In [25]:
Sub_Membership = df_raw[df_raw['product_title'].str.contains('Membership')]
Sub_Membership['product_title'].unique()

array(['Xbox Live 24 Month Gold Membership - Xbox 360 Digital Code',
       'Disney Club Penguin 12 Month Membership Code with Free Amazon Exclusive Bonus Item',
       'Disney Club Penguin Membership',
       'Xbox Live 3 month Gold Membership + 1 bonus month [Online Game Code]',
       '1 Year Membership: AdventureQuest Worlds [Instant Access]',
       '3 Month Membership: AdventureQuest Worlds [Instant Access]',
       'Xbox LIVE 12 Month Gold Membership + 1 Bonus Month - Xbox 360 Digital Code',
       '6 Month Membership: AdventureQuest Worlds [Instant Access]'],
      dtype=object)

In [26]:
Sub_Card = df_raw[df_raw['product_title'].str.contains('Card')]
Sub_Card['product_title'].unique()

array(['Xbox Live Gift Card', 'Playstation Network Card',
       'Xbox 360 Live Points Card', 'Grand Theft Auto V Cash Cards',
       'Hoyle Card Games 2012 AMR',
       'Final Fantasy XIV Online: 60 Day Time Card [Online Game Code]',
       'Hoyle Card Games  [Download]',
       'Xbox $5 Gift Card - Xbox 360 Digital Code',
       "Hoyle Kid's Card Games [Download]",
       'Legends of Solitaire: The Lost Cards [Download]',
       'Xbox $15 Gift Card (Call of Duty Ghosts:\xa0Onslaught DLC) - Xbox 360 Digital Code',
       'Xbox Live 3-Month Gold + $100 Xbox Gift Card - Xbox 360 Digital Code',
       'Xbox Live 3-Month Gold + $5 Xbox Gift Card - Xbox 360 Digital Code',
       'Tripeaks Solitaire Multi (Tripeaks with Multiple Card-Sets and Multiple Layouts) [Download]',
       'Reel Deal Card Games 2011 [Download]',
       '2,013 Card, Mahjongg & Solitaire Games [Download]',
       'Hoyle Card Games [Mac Download]',
       'Sony Playstation Network Card - $10 [Online Game Code]',
       

In [27]:
Sub_Network = df_raw[df_raw['product_title'].str.contains('Network')]
Sub_Network['product_title'].unique()

array(['Playstation Network Card',
       'Sony Playstation Network Card - $10 [Online Game Code]'],
      dtype=object)

In [28]:
df_raw['product_category'].nunique()

1

In [30]:
df_raw['star_rating'].unique()

array([2, 5, 4, 1, 3])

In [33]:
df_raw['review_date'] = pd.to_datetime(df_raw['review_date'])
df_raw['review_date']

0        2015-08-31
1        2015-08-31
2        2015-08-31
3        2015-08-31
4        2015-08-31
            ...    
144719   2008-12-25
144720   2008-12-24
144721   2008-09-10
144722   2008-09-01
144723   2006-08-08
Name: review_date, Length: 144724, dtype: datetime64[ns]

Reviews go from 2006 to 2015

## Data Preparation

Droping all rows that have strings: 
Subscription, Month, Membership,Network on the `product_title` column

In [106]:
df_raw_noSub = df_raw[df_raw["product_title"].str.contains("Subscription|Month|month|Membership|Network|Season Pass") == False]

For titles that have string 'Card' in the title, i have to be more careful since there are some game that include that on their name

In [107]:
Sub_Card2 = df_raw_noSub[df_raw_noSub['product_title'].str.contains('Card')]
Sub_Card2['product_title'].unique()

array(['Xbox Live Gift Card', 'Xbox 360 Live Points Card',
       'Grand Theft Auto V Cash Cards', 'Hoyle Card Games 2012 AMR',
       'Final Fantasy XIV Online: 60 Day Time Card [Online Game Code]',
       'Hoyle Card Games  [Download]',
       'Xbox $5 Gift Card - Xbox 360 Digital Code',
       "Hoyle Kid's Card Games [Download]",
       'Legends of Solitaire: The Lost Cards [Download]',
       'Xbox $15 Gift Card (Call of Duty Ghosts:\xa0Onslaught DLC) - Xbox 360 Digital Code',
       'Tripeaks Solitaire Multi (Tripeaks with Multiple Card-Sets and Multiple Layouts) [Download]',
       'Reel Deal Card Games 2011 [Download]',
       '2,013 Card, Mahjongg & Solitaire Games [Download]',
       'Hoyle Card Games [Mac Download]',
       '1-Year PS Plus + $10 PS Gift Card - PS3 / PS4 [Digital Code]',
       'Five Card Deluxe [Download]',
       '1-Year PS Plus + $50 PS Gift Card - PS3 / PS4 [Digital Code]',
       'Xbox Live $6 Gift Card - Xbox 360 Digital Code',
       '1-Year PS Plus + $

First i will drop rows where 'Gift Card' is found, then i will proceed with other common combinations of words like 'Xbox Live' and Live Points Card

In [108]:
df_raw_noSub2 = df_raw_noSub[df_raw_noSub["product_title"].str.contains("Gift Card|Xbox Live|Live Points Card|Xbox Live Gift Card|Grand Theft Auto V Cash Cards") == False]

In [109]:
Sub_Card3 = df_raw_noSub2[df_raw_noSub2['product_title'].str.contains('Card')]
Sub_Card3['product_title'].unique()

array(['Hoyle Card Games 2012 AMR',
       'Final Fantasy XIV Online: 60 Day Time Card [Online Game Code]',
       'Hoyle Card Games  [Download]',
       "Hoyle Kid's Card Games [Download]",
       'Legends of Solitaire: The Lost Cards [Download]',
       'Tripeaks Solitaire Multi (Tripeaks with Multiple Card-Sets and Multiple Layouts) [Download]',
       'Reel Deal Card Games 2011 [Download]',
       '2,013 Card, Mahjongg & Solitaire Games [Download]',
       'Hoyle Card Games [Mac Download]', 'Five Card Deluxe [Download]',
       "King's Collection: 6 Classic Card Games",
       '8 Card Game Pack [Download]', '5 Realms of Cards [Download]',
       '5 Card Slingo [Download]', 'Card Crazy! [Download]',
       'Rift 30 Day Game Time Card [Online Game Code]',
       'Bicylce Family Card Games [Download]',
       'Rift 60 Day Game Time Card [Online Game Code]',
       'High Stakes Poker: Connelly Card Club [Online Game Code]',
       'Strange Cases: The Tarot Card Mystery [Download]'], dt

Seems like items left with string card in the title are actual games

<br>

Lets look for DLCs

In [110]:
DLC = df_raw_noSub2[df_raw_noSub2['product_title'].str.contains('DLC')]
DLC['product_title'].nunique()

212

In [111]:
len(df_raw_noSub2[df_raw_noSub2['product_title'].str.contains('DLC')])

1698

We have 215 different dlcs, and 1721 reviews, an original copy of the first game is required to be able to run a DLC, I will drop these too.

In [116]:
df_raw_noSub_no_DLC = df_raw_noSub2[df_raw_noSub2["product_title"].str.contains("DLC|Pack") == False]

In [117]:
df_raw_noSub_no_DLC['product_title'].nunique()

6209

In [118]:
df_raw_noSub_no_DLC['product_id'].nunique()

7085

Quantity for product id and product title still doesn't match

In [119]:
sorted(df_raw_noSub_no_DLC['product_title'].unique())

['007 Legends [Download]',
 '1 Moment Of Time: Silentville [Download]',
 '1 PLEX: EVE Online [Instant Access]',
 '1 Penguin 100 Cases [Download]',
 '1 vs 100 [Download]',
 '10 Talismans [Download]',
 '100 % Hidden Objects 2 [Download]',
 '100% Hidden Object (Mac) [Download]',
 '100% Hidden Objects',
 '1001 Japanese Crosswords',
 '1001 Kidz Games [Download]',
 '1001 Mini-Golf Challenge [Download]',
 '1001 Nights: The Adventures of Sindbad',
 '1001 Tangram Puzzles',
 '101 - in - 1 Megamix [Online Game Code]',
 '1080° Snowboarding [Online Game Code]',
 '12 Labours of Hercules 3: Girl Power [Download]',
 '12 Labours of Hercules II: The Cretan Bull [Download]',
 '12 Labours of Hercules [Download]',
 '12 PLEX: EVE Online [Instant Access]',
 '12000 AC plus Game Booster Addon: AdventureQuest Worlds [Game Connect]',
 '15 Square Slider Puzzle [Download]',
 '16 Bit Arena [Download]',
 '18 Wheels of Steel American Long Haul [Download]',
 '18 Wheels of Steel American Long Haul [Online Game Code]',


Some titles include a description next to the title, with the way buyers could access the game, for example:

<br>

`18 Wheels of Steel American Long Haul [Download]`

`18 Wheels of Steel American Long Haul [Online Game Code]`

<br>

Definitely the same game, just different "delivery" method.


In [76]:
df_raw_noSub_no_DLC[df_raw_noSub_no_DLC['product_title'].str.contains('18 Wheels of Steel American Long Haul')].head(3)

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
39224,US,36420328,R2LOZS1XJTWZNN,B004GHNG0E,614285704,18 Wheels of Steel American Long Haul [Download],Digital_Video_Games,4,0,0,N,Y,Four Stars,ok,2014-10-15
50242,US,17845931,R2TJLHZ9X7EG83,B004GHNG0E,614285704,18 Wheels of Steel American Long Haul [Download],Digital_Video_Games,4,0,0,N,Y,Four Stars,Like driving trucks.,2014-07-14
74152,US,21374226,R388BHLY4GIU5E,B00CLVZGPK,775610170,18 Wheels of Steel American Long Haul [Online ...,Digital_Video_Games,1,1,1,N,Y,"Doesn't work, no manufacturer support",The code I got does not work. Amazon referred ...,2013-12-29



<br>

The `[Download]` edition of the game, on the second row above, has a different `product_id` than its `[Online Game Code]` edition (on the 3rd row above), it's the same game but taged with different ID

<br>


At first sight, `[Instant Access]`, `[Download]`, `[Game Connect]` and `[Online Game Code]` seem to be the tags at the end of the games titles, if delete those out of the title name, i could use them so the rec system instead of the id, which was my first thought

In [143]:
delivery_type = [' [Instant Access]', ' [Download]', ' [Game Connect]', ' [Online Game Code]']

In [198]:
clean_product_title = []
for x in df_raw_noSub_no_DLC['product_title']:
    for y in delivery_type:
        if y in x:
            clean_product_title.append(x.replace(y, ''))
            
        elif y not in x:
            if x not in clean_product_title:
                clean_product_title.append
        
            
            

In [ ]:
clean_product_title.append(x.replace(y, ''))

In [200]:
clean_product_title

['Madden NFL 16 - Xbox One Digital Code',
 'Command & Conquer The Ultimate Collection',
 'Command & Conquer The Ultimate Collection [Instant Access]',
 'Saints Row IV - Enter The Dominatrix [Online Game Code]',
 'Saints Row IV - Enter The Dominatrix',
 'Double Dragon: Neon [Online Game Code]',
 'Double Dragon: Neon',
 'Sims 4',
 "Sid Meier's Civilization V",
 'Build-a-lot 4: Power Source [Download]',
 'Build-a-lot 4: Power Source',
 'Borderlands 2',
 'Minecraft for PC/Mac [Online Game Code]',
 'Minecraft for PC/Mac',
 "Brink of Consciousness: Lonely Hearts Murders - Collector's Edition",
 'Nat Geo Adventure: Eco Rescue Rivers [Download]',
 'Nat Geo Adventure: Eco Rescue Rivers',
 'Fable III',
 'PES 2015 - Parent',
 'Might & Magic X Legacy Deluxe Edition [Download]',
 'Might & Magic X Legacy Deluxe Edition',
 'Batman: Arkham Knight',
 'Star Wars The Old Republic',
 'BlazBlue: Calamity Trigger [Online Game Code]',
 'BlazBlue: Calamity Trigger',
 'Destiny - Expansion Pass - PlayStation 4 

In [193]:
df_raw_noSub_no_DLC['product_title']

0                     Madden NFL 16 - Xbox One Digital Code
2         Command & Conquer The Ultimate Collection [Ins...
4         Saints Row IV - Enter The Dominatrix [Online G...
5                    Double Dragon: Neon [Online Game Code]
6                                                    Sims 4
                                ...                        
144719                          Crazy Machines 2 [Download]
144720    Crazy Machines 1 - The Wacky Contraptions Game...
144721                          Crazy Machines 2 [Download]
144722                          Crazy Machines 2 [Download]
144723                           Emperor of the Fading Suns
Name: product_title, Length: 101276, dtype: object

In [120]:
df_raw_noSub_no_DLC['customer_id'].nunique()

77038

In [65]:
df_raw_noSub_no_DLC

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,21269168,RSH1OZ87OYK92,B013PURRZW,603406193,Madden NFL 16 - Xbox One Digital Code,Digital_Video_Games,2,2,3,N,N,A slight improvement from last year.,I keep buying madden every year hoping they ge...,2015-08-31
2,US,45765011,R3YOOS71KM5M9,B00DNHLFQA,951665344,Command & Conquer The Ultimate Collection [Ins...,Digital_Video_Games,5,0,0,N,Y,Hail to the great Yuri!,If you are prepping for the end of the world t...,2015-08-31
4,US,22151364,RV2W9SGDNQA2C,B00G9BNLQE,640460561,Saints Row IV - Enter The Dominatrix [Online G...,Digital_Video_Games,5,0,0,N,Y,Five Stars,Awesome!,2015-08-31
5,US,22151364,R3CFKLIZ0I2KOB,B00IMIL498,621922192,Double Dragon: Neon [Online Game Code],Digital_Video_Games,5,0,0,N,Y,Five Stars,Awesome!,2015-08-31
6,US,38426028,R1LRYU1V0T3O38,B00S00IJH8,215163395,Sims 4,Digital_Video_Games,4,0,0,N,Y,i like the new skills like herbalism in this,"i like the new skills like herbalism in this, ...",2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144719,US,53011810,R2G7DI8NYXZB5R,B001AUEITS,163061733,Crazy Machines 2 [Download],Digital_Video_Games,4,2,3,N,N,Worked first try for me,"I was worried due to the 2 reviews I saw here,...",2008-12-25
144720,US,53094564,R3QRKP4DS759BP,B001AU6TQ8,801870836,Crazy Machines 1 - The Wacky Contraptions Game...,Digital_Video_Games,1,13,16,N,N,"The Software May be Great, But I'll Never Know",I downloaded this as a Christmas present for m...,2008-12-24
144721,US,37181147,R24K4C0ZC3093U,B001AUEITS,163061733,Crazy Machines 2 [Download],Digital_Video_Games,3,3,3,N,N,Some install problems but good otherwise,The previous reviewer is correct in noting tha...,2008-09-10
144722,US,18614365,R13OA3TRCM8IBM,B001AUEITS,163061733,Crazy Machines 2 [Download],Digital_Video_Games,1,20,22,N,N,Do Not Download This!,I downloaded this for my son's birthday yester...,2008-09-01


In [ ]:
from surprise.model_selection import train_test_split

In [ ]:
trainset, testset = train_test_split(jokes, test_size=0.2)

## Modeling

In [ ]:
from surprise.prediction_algorithms import SVD
from surprise.model_selection import GridSearchCV

## Evaluation

In [ ]:
making predictions

## Deployment

## metadata

In [2]:
!ls data


Digital_Video_Games.tsv  meta_Video_Games.json.gz


In [3]:
import gzip

In [6]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

meta_df = getDF('./data/meta_Video_Games.json.gz')